# Project Milestone Two: Modeling and Feature Engineering

### Due: Midnight on April 13 (with 2-hour grace period) and worth 25 points

### Overview

This milestone builds on your work from Milestone 1. You will:

1. Evaluate baseline models using default settings.
2. Engineer new features and re-evaluate models.
3. Use feature selection techniques to find promising subsets.
4. Select the top 3 models and fine-tune them for optimal performance.

You must do all work in this notebook and upload to your team leader's account in Gradescope. There is no
Individual Assessment for this Milestone. 


In [1]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [3]:
# ===================================
# Useful Imports: Add more as needed
# ===================================

# Standard Libraries
import os
import time
import math
import io
import zipfile
import requests
from urllib.parse import urlparse
from itertools import chain, combinations

# Data Science Libraries
import numpy as np
import pandas as pd
import seaborn as sns

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as mticker  # Optional: Format y-axis labels as dollars
import seaborn as sns

# Scikit-learn (Machine Learning)
from sklearn.model_selection import (
    train_test_split, 
    cross_val_score, 
    GridSearchCV, 
    RandomizedSearchCV, 
    RepeatedKFold
)
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector, f_regression, SelectKBest
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor

# Progress Tracking

from tqdm import tqdm

# =============================
# Global Variables
# =============================
random_state = 42

# =============================
# Utility Functions
# =============================

# Format y-axis labels as dollars with commas (optional)
def dollar_format(x, pos):
    return f'${x:,.0f}'

# Convert seconds to HH:MM:SS format
def format_hms(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds))



In [ ]:
#this is my change 

## Prelude: Load your Preprocessed Dataset from Milestone 1

In Milestone 1, you handled missing values, encoded categorical features, and explored your data. Before you begin this milestone, you’ll need to load that cleaned dataset and prepare it for modeling.

Here’s what to do:

1. Return to your Milestone 1 notebook and rerun your code through Part 3, where your dataset was fully cleaned (assume it’s called `df_cleaned`).

2. **Save** the cleaned dataset to a file by running:

>   df_cleaned.to_csv("zillow_cleaned.csv", index=False)

3. Switch to this notebook and **load** the saved data:

>   df = pd.read_csv("zillow_cleaned.csv")

4. Create a **train/test split** using `train_test_split`.  
   
6. **Standardize** the features (but not the target!) using **only the training data.** This ensures consistency across models without introducing data leakage from the test set:

>   scaler = StandardScaler()   
>   X_train_scaled = scaler.fit_transform(X_train)    
  
**Notes:** 

- You will not use the testing set during this milestone — it’s reserved for final evaluation later.
- You will have to redo the scaling step when you introduce new features (which have to be scaled as well).


In [7]:
df_cleaned = pd.read_csv('zillow_cleaned.csv')
print(df_cleaned.describe())
print(df_cleaned.info())


       airconditioningtypeid  architecturalstyletypeid   bathroomcnt  \
count           77613.000000              77613.000000  77613.000000   
mean                1.812013                  7.386473      2.298496   
std                 1.683429                  0.140546      0.996513   
min                 1.000000                  2.000000      0.000000   
25%                 1.000000                  7.386473      2.000000   
50%                 1.812013                  7.386473      2.000000   
75%                 1.812013                  7.386473      3.000000   
max                13.000000                 21.000000     18.000000   

         bedroomcnt  buildingclasstypeid  buildingqualitytypeid  \
count  77613.000000         77613.000000           77613.000000   
mean       3.053223             3.933333               6.533779   
std        1.140230             0.003468               1.379538   
min        0.000000             3.000000               1.000000   
25%        2.000

In [8]:
# missing_data = df.isnull().sum().sort_values(ascending=False)

# # Impute missing numerical values with the mean and categorical values with the mode
# numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
# categorical_cols = df.select_dtypes(include=['object']).columns

# # Insert numerical columns with the mean
# df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].mean())

# # Insert categorical columns with the mode
# df[categorical_cols].fillna(df[categorical_cols].mode().iloc[0])

# # One-Hot Encoding categorical variables
# df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

In [9]:
# drop_cols_1 = ['basementsqft', 'yardbuildingsqft17', 'fireplacecnt', 'storytypeid']

# # Drop the columns
# df.drop(columns=drop_cols_1, inplace=True)

# # Confirm the changes
# df.info()

In [10]:
# # drop samples with too many null values 
# threshold = 0.50  #gets rid of rows with over 50% missing values 
# null_percentage = df.isnull().mean(axis=1)
# df_cleaned3c = df[null_percentage < threshold]
# #print(df_cleaned3c) #these is the dataframe with dropped samples with na values > 50%

# # drop samples with null values in target 
# df_no_null_target = df_cleaned3c[df_cleaned3c['taxvaluedollarcnt'].notnull()]
# #print(df_no_null_target)

# #get rid of samples with outliers in target using quartiles
# q1 = df_no_null_target['taxvaluedollarcnt'].quantile(0.25)
# q3 = df_no_null_target['taxvaluedollarcnt'].quantile(0.75)
# IQR = q3 - q1
# lower_bound = q1 - 1.5 * IQR
# upper_bound = q3 + 1.5 * IQR

# df_no_target_outliers = df_no_null_target[(df_no_null_target['taxvaluedollarcnt'] >= lower_bound) & (df_no_null_target['taxvaluedollarcnt']<= upper_bound)]
# print(df_no_target_outliers)
# df_no_target_outliers.info()

# # mode imputation since categorical variables present

# mode_imputed_data = df_no_target_outliers.fillna(df_no_target_outliers.mode().iloc[0])
# # print(mode_imputed_data)

# df_cleaned = mode_imputed_data
# print(df_cleaned)

# cleaned_data = pd.DataFrame(df_cleaned)
# cleaned_data.to_csv('M1_cleaned_data.csv', index=False)

### Part 1: Baseline Modeling [3 pts]

Apply the following regression models to the scaled training dataset using **default parameters**:

- Linear Regression
- Ridge Regression
- Lasso Regression
- Decision Tree Regression
- Bagging
- Random Forest
- Gradient Boosting Trees

For each model:
- Use **repeated cross-validation** (e.g., 5 folds, 5 repeats).
- Report the **mean and standard deviation of CV RMSE Score** across all folds in a table. 


In [17]:
# Add as many code cells as you need

### Part 1: Discussion [2 pts]

In a paragraph or well-organized set of bullet points, briefly compare and discuss:

  - Which models perform best overall?
  - Which are most stable (lowest std)?
  - Any signs of overfitting or underfitting?

> Your text here

### Part 2: Feature Engineering [3 pts]

Consider **at least three new features** based on your Milestone 1, Part 5. Examples include:
- Polynomial terms
- Log or interaction terms
- Groupings or transformations of categorical features

Add these features to `X_train` and then:
- Scale using `StandardScaler` 
- Re-run all models listed above (using default settings again).
- Report updated RMSE scores (mean and std) across repeated CV in a table. 

**Note:**  Recall that this will require creating a new version of the dataset, so effectively you may be running "polynomial regression" using `LinearRegression`. 

In [ ]:
# Add as many code cells as you need

from sklearn.preprocessing import StandardScaler, PolynomialFeatures 
from sklearn.model_selection import cross_val_score, RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


In [19]:
# train test split cleaned dataset 
X = df_cleaned.drop(columns=['taxvaluedollarcnt']) # Features (all columns except the target)
y = df_cleaned['taxvaluedollarcnt'] # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_num = X_train.select_dtypes(include=[np.number])
X_test_num = X_test.select_dtypes(include=[np.number])

In [22]:
# Feature 1: Polynomial Features 

#setup poly 
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_num)
X_test_poly = poly.transform(X_test_num)

feature_names_poly = poly.get_feature_names_out(X_train_num.columns)
#train values on poly 
X_train_poly_df_cleaned = pd.DataFrame(X_train_poly, columns=feature_names_poly, index=X_train.index)
X_test_poly_df_cleaned = pd.DataFrame(X_test_poly, columns=feature_names_poly, index=X_test.index)
#print(X_train_poly_df_cleaned)

#scale  
scaler = StandardScaler()
X_train_poly_scaled = scaler.fit_transform(X_train_poly_df_cleaned)
X_test_poly_scaled = scaler.transform(X_test_poly_df_cleaned)

In [ ]:
# Feature 1 Modeling: 




In [ ]:
# Feature 2: Logarithmic Features 

#setup log
X_train_log = np.log1p(X_train_num)
X_test_log = np.log1p(X_test_num)

X_train_log.columns = [f'log_{col}' for col in X_train_log.columns]
X_test_log.columns = [f'log_{col}' for col in X_test_log.columns]

#scale 
scaler = StandardScaler()
X_train_log_scaled = scaler.fit_transform(X_train_log)
X_test_log_scaled = scaler.fit_transform(X_test_log)


/home/codespace/.local/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log1p
  result = func(self.values, **kwargs)
/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/codespace/.local/lib/python3.12/site-packages/sklearn/ut

In [ ]:
# Feature 2 Modeling: 



In [25]:
pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 45.1 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [29]:
# Feature 3: Target Encoding

#setup Target Encoding
import category_encoders as ce
categorical_cols = X_train.select_dtypes(include='object').columns
target_encoder = ce.TargetEncoder(cols=categorical_cols)
X_train_target_encoder = target_encoder.fit_transform(X_train[categorical_cols], y_train)
X_test_target_encoder = target_encoder.transform(X_test[categorical_cols])




In [ ]:
# Feature 3 Modeling: 




In [30]:
# combine all features into X_train

X_train_combined = pd.concat([
    pd.DataFrame(X_train_poly_df_cleaned, columns=feature_names_poly, index=X_train.index),
    X_train_log, 
    X_train_target_encoder
], axis=1)

X_test_combined = pd.concat([
    pd.DataFrame(X_test_poly_df_cleaned, columns=feature_names_poly, index=X_test.index), 
    X_test_log, 
    X_test_target_encoder
], axis=1)

In [31]:
#scale using StandardScaling

scaler = StandardScaler()
X_train_scaled_combined = scaler.fit_transform(X_train_combined)
X_test_scaled_combiend = scaler.fit_transform(X_test_combined)

: 

### Part 2: Discussion [2 pts]

Reflect on the impact of your new features:

- Did any models show notable improvement in performance?

- Which new features seemed to help — and in which models?

- Do you have any hypotheses about why a particular feature helped (or didn’t)?

- Were there any unexpected results?



> Your text here

### Part 3: Feature Selection [3 pts]

Using the full set of features (original + engineered):
- Apply **feature selection** methods to investigate whether you can improve performance.
  - You may use forward selection, backward selection, or feature importance from tree-based models.
- For each model, identify the **best-performing subset of features**.
- Re-run each model using only those features.
- Report updated RMSE scores (mean and std) across repeated CV in a table.


### Part 3: Discussion [2 pts]

Analyze the effect of feature selection on your models:

- Did performance improve for any models after reducing the number of features?

- Which features were consistently retained across models?

- Were any of your newly engineered features selected as important?

- How did feature selection differ between linear and tree-based models?

> Your text here

### Part 4: Fine-Tuning Your Top 3 Models [6 pts]

In this final phase of Milestone 2, you’ll select and refine your **three most promising models and their corresponding data pipelines** based on everything you've done so far.

1. Choose the top 3 models based on performance and interpretability from earlier parts.
2. For each model:
   - Perform hyperparameter tuning using `sweep_parameters`, `GridSearchCV`, `RandomizedSearchCV`, or other techniques from previous homeworks. 
   - Experiment with different versions of your feature engineering and preprocessing — treat these as additional tunable components.
3. Report the mean and standard deviation of CV RMSE score for each model in a summary table.



In [19]:
# Add as many code cells as you need

### Part 4: Discussion [4 pts]

Reflect on your tuning process and final results:

- What was your tuning strategy for each model? Why did you choose those hyperparameters?
- Did you find that certain types of preprocessing or feature engineering worked better with specific models?
- Provide a ranking of your three models and explain your reasoning — not just based on RMSE, but also interpretability, training time, or generalizability.
- Conclude by considering whether this workflow has produced the results you expected. Typically, you would repeat steps 2 - 4 and also reconsider the choices you made in Milestone 1 when cleaning the dataset, until reaching the point of diminishing returns; do you think that would that have helped here?

> Your text here